# Documentacion de Pandas

https://pandas.pydata.org/docs/user_guide/10min.html

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('archive.zip', compression='zip', header=0, sep=',', quotechar='"')

In [ ]:
df.sample(n=20, random_state=1)

In [ ]:
df.sample(n=20, random_state=1, weights='price')

In [ ]:
df.head()
df.info(memory_usage="deep")

In [ ]:
df.loc[~df.license.isnull()]

In [ ]:
#low memory solo funciona con el engine de C para numpy.  Por defecto pandas usa pyarrow.  Tambien esta el motor de python
#que es mas flexible pero mas lento
#Pandas no soporta fechas en su casting desde csv, hay que user el parametro parse_dates para indicar que columnas son fechas
parse_dates=['last review']
dtypes = {'license': str}
df = pd.read_csv('archive.zip', compression='zip', header=0, sep=',', quotechar='"', dtype=dtypes, parse_dates=parse_dates)
df.head()

In [ ]:
df.info(memory_usage="deep")

In [ ]:
#Conteo de nulls por columna
df_nulls = df.isnull().sum().sort_values(ascending=False).to_frame('count')
#Conteo total de filas
df_nulls['total'] = df.shape[0]
#% de columnas vacias
df_nulls['%'] = df_nulls['count']/df_nulls['total']
df_nulls

In [ ]:
df.drop('license', axis=1, inplace=True)

In [ ]:
df.info(memory_usage="deep")

In [ ]:
#Revisar los outliers con un simple min y max
df.agg([min, max])

In [ ]:
df['price'] = pd.to_numeric(df['price'].str.replace('$','', regex=False), errors='coerce')

In [ ]:
df['service fee'] = pd.to_numeric(df['service fee'].str.replace('$','', regex=False), errors='coerce')

In [ ]:
df.agg([min, max])

In [ ]:
df[df['last review'].apply(lambda x: x.year) > 2022]

In [ ]:
df = df[df['last review'].apply(lambda x: x.year) <= 2022]
df

In [ ]:
#Revisar duplicados
df.duplicated().sum()

In [ ]:
df.duplicated(subset=['host name', 'lat', 'long']).sum()

In [ ]:
df.drop_duplicates(subset=['host name', 'lat', 'long'], inplace=True)

In [ ]:
df.info(memory_usage="deep")

In [ ]:
df['country'].unique()

In [ ]:
df['country code'].unique()

In [ ]:
df['neighbourhood group'].unique()

In [ ]:
df.loc[df['country'].isnull(), 'country'] = 'United States'
df.loc[df['country code'].isnull(), 'country code'] = 'US'

In [ ]:
df_countries = pd.read_csv('countries.csv', header=0, sep=',', quotechar='"')
df_countries

In [ ]:
df_countries.drop('English short name lower case', axis=1, inplace=True)
df_countries

In [ ]:
#unir ambos datasets
merged_df = pd.merge(df, df_countries, left_on='country code', right_on='Alpha-2 code',how="inner")
merged_df

In [ ]:
df.groupby(by=['country']).mean()

In [ ]:
df.groupby(by=['neighbourhood group']).mean()

In [ ]:
df.groupby(by=['neighbourhood group'])['service fee'].mean()

In [ ]:
df.loc[df['neighbourhood group']=='brookln', 'neighbourhood group'] = 'Brooklyn'
df.loc[df['neighbourhood group']=='manhatan', 'neighbourhood group'] = 'Manhattan'
df.groupby(by=['neighbourhood group'])['service fee'].mean()

In [ ]:
df.groupby(by=['neighbourhood group']).agg({'service fee':{ 'mean', 'min', 'max' }})